In [8]:
import os
import math
from datetime import datetime, timedelta

from helpers import general_helpers
import pytest 
import pytz

project_home = os.environ['PACKAGE_HOME']
import configparser
config = configparser.ConfigParser()
config.read(project_home + '/Development/config.ini')
start_date = config['DATES']['START_DATE']
end_date = config['DATES']['END_DATE']



rawxml_folder = '{}/raw_data'.format(config['FOLDERS']['WORKING_FOLDER'])
cleanxml__folder = '{}/raw_data'.format(config['FOLDERS']['WORKING_FOLDER'])
parsedxml__folder = '{}/raw_data'.format(config['FOLDERS']['WORKING_FOLDER'])

In [9]:
qa_db = 'patent_QA'
new_db = config['DATABASE']['NEW_DB']
db_con = general_helpers.connect_to_db(config['DATABASE']['HOST'], config['DATABASE']['USERNAME'], config['DATABASE']['PASSWORD'], qa_db)

In [14]:
# REQUIRES: start and end timestamps (database format "YYYYMMDD")
# MODIFIES: nothing 
# EFFECTS: returns number of weeks data update spans
def get_num_weeks(start_timestamp, end_timestamp):
    
    time_period = end_timestamp - start_timestamp
    num_weeks = math.ceil(time_period.days // 7)
    
    return num_weeks

# REQUIRES: data update start & end dates (from config file, database format "YYYYMMDD")
# MODIFIES: nothing 
# EFFECTS: returns number of weeks data update spans, all tuesdays of data
def get_tuesdays(start_date, end_date):
    
    # convert dates to datetime objects
    start_timestamp = datetime.strptime(start_date,'%Y%m%d')
    end_timestamp = datetime.strptime(end_date,'%Y%m%d')

    # see how many weeks data spans ( get # of files that should be downloaded)
    num_weeks = get_num_weeks(start_timestamp, end_timestamp)
    
    # now get dates of all tuesdays part of data update
    all_tuesdays = []

    # need to -1 from start timestamp since it will always be a wednesday
    ts = start_timestamp - timedelta(days=1)

    for i in range(0, num_weeks):
        # get date of next tuesday
        ts = ts + timedelta(days=7)
        ts_str = datetime.strftime(ts, '%Y%m%d')
        all_tuesdays.append(ts_str)
        
    return num_weeks, all_tuesdays


# REQUIRES: data folder path, num_weeks
# MODIFIES: nothing
# EFFECTS: asserts if # of folders matches expected # for data update
def test_num_xmlfiles(db_con, folder_path, num_weeks, file_type):
    
    num_xml_files = len(os.listdir(folder_path))
    
    # static
    stmt_pt1 = "insert into {0}.test_num_xmlfiles (`db_timestamp`, `type`, `expected_value`, `actual_value`)".format(qa_db)

    # dynamic
    stmt_pt2 = "values ('{0}', '{1}', {2}, {3});".format(new_db, file_type,num_weeks, num_xml_files)

    # insert row into qa table
    insert_stmt = stmt_pt1 + stmt_pt2
    db_con.execute(insert_stmt)
      
    assert(num_xml_files == num_weeks)

# REQUIRES: raw xml folder path, num_weeks
# MODIFIES: nothing
# EFFECTS: returns 
def test_size_xml(folder_path, num_weeks):
    
    xml_files = os.listdir(folder_path)
        

    if(len(xml_files) == num_weeks):
        #print('Pass: Expected number of xml files are present.')
        bool_xml = True
        
    else: 
        print('Fail: expected number of xml files are not present.')
        
    return bool_xml

# REQUIRES: folder path, list of files to check
# MODIFIES: nothing
# EFFECTS: returns true or false for checking bulk file downloads
# def check_filesize(folder_path, files_to_check):



In [15]:
#1. Get span of data update
num_weeks, all_tuesdays = get_tuesdays(start_date, end_date)

print("Number of weeks of data update: ", num_weeks)
print("Tuesdays are: ", all_tuesdays)

#2. Check raw XML files were downloaded
test_num_xmlfiles(db_con, rawxml_folder, num_weeks, 'raw xml')

#3. Check that raw XML files have size > 0MB

#4. Check that clean XML files were created

#5. Check that clean XML files have size > 0MB

#6. Check that a parsed XML folder was created for each Tuesday

#7. Check that each expected .csv is in each parsed folder 

#8. Check that each parsed data file has a size > 0 MB


Number of weeks of data update:  6
Tuesdays are:  ['20190827', '20190903', '20190910', '20190917', '20190924', '20191001']


AssertionError: 

In [ ]:


db_con.execute("insert into {0}.test_num_xmlfiles (`db_timestamp`, `expected_value`, `actual_value`) values ('{1}', 5, 4);".format(qa_db, new_db))

In [ ]:
all_tuesdays

In [ ]:

downloaded_xml_files = os.listdir(rawdata_folder)


In [ ]:
downloaded_xml_files

In [ ]:
os.path.getsize(rawdata_folder + '/ipg181127.xml')

os.stat(rawdata_folder).st_size